In [ ]:
第六关：机器学习算法实践实战Baseline（不使用GridSearchCV优化得分为0.74）

大家可以通过学习基础代码，修改下面数据处理部分、模型调参等部分的代码，提高准确率
具体修改的地方可以查看🚩标识处建议，可以使用Ctrl+f（Mac用户：command+f）查找

在前面几关的学习中，总共学习了逻辑回归、KMeans、决策树、SVM 支持向量机和 XGBoost，我相信通过前面实际数据的模型实践和闯关题的解答，大家都已经学会了如何训练模型，那么我们就开始这个训练营的最后一关 堪培拉天气数据预测实战。

下面我会给大家列出具体的实施步骤，需要大家补充完代码然后提交文件！大家可以参考之前的关卡完成实战项目。

开始之前我先介绍下实战的数据集：

train_weather.csv 训练数据集。
test_weather.csv 提交测试数据集。
submit_result.csv 提交结果数据集。
数据集中的特征含义表示如下：

特征列名称	特征含义
Date	日期
Location	观察的城市
MinTemp	当天最低温度（摄氏度）
MaxTemp	当天最高温度（摄氏度）温度都是 string
Rainfall	当天的降雨量（单位是毫米mm）
Evaporation	一个凹地上面水的蒸发量（单位是毫米mm），24小时内到早上9点
Sunshine	一天中出太阳的小时数
WindGustDir	最强劲的那股风的风向，24小时内到午夜
WindGustSpeed	最强劲的那股风的风速（km/h），24小时内到午夜
WindDir9am	上午9点的风向
WindDir3pm	下午3点的风向
WindSpeed9am	上午9点之前的十分钟里的平均风速，即 8:50~9:00的平均风速，单位是（km/hr）
WindSpeed3pm	下午3点之前的十分钟里的平均风速，即 14:50~15:00的平均风速，单位是（km/hr）
Humidity9am	上午9点的湿度
Humidity3pm	下午3点的湿度
Pressure9am	上午9点的大气压强（hpa）
Pressure3pm	下午3点的大气压强
Cloud9am	上午9点天空中云的密度，取值是[0, 8]，以1位一个单位，0的话表示天空中几乎没云，8的话表示天空中几乎被云覆盖了
Cloud3pm	下午3点天空中云的密度
Temp9am	上午9点的温度（单位是摄氏度）
Temp3pm	下午3点的温度（单位是摄氏度）
RainTomorrow	明天是否下雨标签
引入依赖
如果你使用的机器学习方法为下面备注里的，则不用修改，否则请添加对应的方法

In [ ]:

# 引入依赖

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression # 逻辑回归
from sklearn.tree import DecisionTreeClassifier     # 决策树
from sklearn.ensemble import RandomForestClassifier # 随机森林
from sklearn.svm import SVC                         # 支持向量机
from sklearn.cluster import KMeans                  # Kmeans 
from xgboost import XGBClassifier                   # XGBoost
from sklearn.metrics import accuracy_score

In [ ]:
加载数据（不用修改）

In [ ]:
# 加载数据

train = pd.read_csv('/home/mw/input/canberraWeather2321/train_weather.csv', index_col='Unnamed: 0')
test = pd.read_csv('/home/mw/input/canberraWeather2321/test_weather.csv', index_col='Unnamed: 0')
submit = pd.read_csv('/home/mw/input/canberraWeather2321/submit_result.csv', index_col='Unnamed: 0')


# 查看数据集，了解数据集格式
print("训练集：\n")
print(train.head())
print("测试集：\n")
print(test.head())
print("提交集：\n")
print(submit.head())

In [ ]:
数据基础性分析

In [ ]:
# 查看训练集和结果集数据空值、类型
print("========训练集=======\n")
print(train.info())
print("========测试集=======\n")
print(test.info())

In [ ]:
# 训练集和结果集基础分析查看（是否有非法数据）
# 🔔提示❗️❗️🔔：训练集与测试集都有非法数据，注意表格里数据的范围与题目要求的数据范围

print("=== 训练集数值特征统计 ===")
display(train.describe().T)

print("=== 测试集数值特征统计 ===")
display(test.describe().T)

In [ ]:
异常数据处理

In [ ]:
# 非法值处理，训练集和测试集中均存在非法值，删除对应数据的索引 index（测试集中若出现非法值，提交答案集中的响应index 也需要删除）
# 提示🔔：
# 步骤一1️⃣：找到非法值的索引
test[test['Cloud9am'] == 9].index, train[train['Cloud3pm'] == 9].index
# 返回的内容为非法值的索引:前为训练集，后为测试集
# (Int64Index([索引], dtype='int64'), Int64Index([索引], dtype='int64'))

In [ ]:
# 步骤二2️⃣：删除对应的索引

train = train.drop([88723])
# submit 与 test 删除相同的索引是因为 test 最终的预测结果会缺失，而submit 也需要一致删除，这俩索引一样
test = test.drop([20890, 22448])
submit = submit.drop([20890, 22448]) # submit 与 test 删除相同的索引是因为 test 最终的预测结果会缺失，而submit 也需要一致删除

In [ ]:
# 更改索引为日期
# 训练集
train.set_index('Date', inplace=True)
# 测试集
test.set_index('Date', inplace=True)

In [ ]:
数据处理，下面的代码可以开始修改了

In [ ]:
# 离散型数据处理缺失值
# 离散型数据则是指只能取到有限个数或者是可数个数的数据，通常以整数表示。
# 🚩：我这里用的是众数，你也可以中位数、平均数等等，想想哪个更好
# 这里需要单独对训练集和测试集分别处理，因为测试集中没有 RainTomorrow 字段

cate_columns = ['RainTomorrow', 'WindDir3pm', 'WindDir9am', 'WindGustDir']

si = SimpleImputer(missing_values=np.nan,strategy="most_frequent") # 使用众数填充缺失值
 
# 步骤1：用训练集拟合规则    
si.fit(train.loc[:, cate_columns])   
# 步骤2：填充训练集缺失值    
train.loc[:, cate_columns] = si.transform(train.loc[:, cate_columns])

In [ ]:
# 测试集
cate_columns = ['WindDir3pm', 'WindDir9am', 'WindGustDir']
# 🚩：我这里用的是众数，你也可以中位数、平均数等等，想想哪个更好
si = SimpleImputer(missing_values=np.nan,strategy="most_frequent") # 使用众数填充缺失值

# 这里需要单独对训练集和测试集分别处理，因为测试集中没有 RainTomorrow 字段
# 步骤1   
si.fit(test.loc[:, cate_columns])
# 步骤2  
test.loc[:, cate_columns] = si.transform(test.loc[:, cate_columns])

In [ ]:
# 连续型数据处理缺失值
# 连续型数据是指可以取到某个区间内的任意值的数据，通常以实数表示。
# 🚩：我这里用的是平均数，你也可以用众数、中位数、等等，想想哪个更好
cate_columns = ['RainTomorrow', 'WindDir3pm', 'WindDir9am', 'WindGustDir', 'Location']

columns = train.columns.to_list()
for col in cate_columns:
    columns.remove(col)

impmean = SimpleImputer(missing_values=np.nan,strategy = "median")
# 步骤1 
impmean = impmean.fit(train.loc[:, columns])
# 步骤2
train.loc[:, columns] = impmean.transform(train.loc[:, columns])

In [ ]:
# 测试集

cate_columns = ['WindDir3pm', 'WindDir9am', 'WindGustDir', 'Location']

columns = test.columns.to_list()
for col in cate_columns:
    columns.remove(col)
# 🚩：我这里用的是平均数，你也可以用众数、中位数、等等，想想哪个更好
impmean = SimpleImputer(missing_values=np.nan,strategy = "median")
# 步骤1 
impmean = impmean.fit(test.loc[:, columns])
# 步骤2
test.loc[:, columns] = impmean.transform(test.loc[:, columns])

In [ ]:
# 正确处理的标志：下面的输出全是'0'
train.isnull().sum(), test.isnull().sum()

In [ ]:
# 离散型数据标准化

cate_columns = ['RainTomorrow', 'WindDir3pm', 'WindDir9am', 'WindGustDir', 'Location']
lb = LabelEncoder()

# 用LabelEncoder将离散特征（如风向、是否下雨）转为数值型，便于模型处理：
for col in cate_columns:
    # 下面三行代码都是打印信息，方便读者查看
    print('-'*50)
    print(train[col].unique())
    print(train[col].value_counts())
    
    # 步骤1：用训练集拟合编码器规则
    lb.fit(train[col])
    
    # 步骤2：使用拟合好的规则转换训练集
    train[col] = lb.transform(train[col])

In [ ]:
# 测试集

cate_columns = ['WindDir3pm', 'WindDir9am', 'WindGustDir', 'Location']
lb = LabelEncoder()

for col in cate_columns:
    # 下面三行代码都是打印信息，方便读者查看
    print('-'*50)
    print(train[col].unique())
    print(train[col].value_counts())
    # 步骤1 
    lb.fit(test[col])
    
    # 步骤2
    test[col] = lb.transform(test[col])

In [ ]:
# 特征相关性

sns.heatmap(train.corr())

In [ ]:
训练模型
你可以使用 逻辑回归、支持向量机、决策树、XGBoost等多种我们学习过的机器学习方法
下面的代码采用随机森林，仅为参考，无法通关本训练营
请你参考下面的代码，自行修改，找到最佳模型与参数

推荐使用 XGBoost
其他算法经测试合理的参数也是可以通关的
训练时间大约在2～8小时，具体时间根据选择的算法不同有所不同

In [ ]:
# 切分训练集和测试集

x = train.drop('RainTomorrow', axis=1)
y = train['RainTomorrow']

# 🚩：你可以自由划分test_size的大小，我这里是0.5，显然不合理，随机数种子也可以自由修改
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# 查看信息
x_train.head(), x_test.head(), y_train.head(), y_test.head()

In [ ]:
# 🚩：构建模型，修改下面的代码，但是不要修改变量名：'model'

'''
model = 模型名称(
    # 模型参数
    # 请你删除注释后在这里修改🚩
)
'''


# 下面代码以随机森林为例
'''
随机森林基于决策树，其主要参数有：
1. 控制树数量的参数：
   n_estimators: 森林中决策树的数量

2. 控制单棵树结构的参数（与决策树共享）：
   max_depth: 每棵树的最大深度。
   min_samples_split: 节点分裂所需的最小样本数。
   min_samples_leaf: 叶节点所需的最小样本数。
   max_features: 分裂时考虑的最大特征数。（默认'sqrt'，即√n）。

3. 计算优化参数：
   n_jobs: 并行训练的CPU核心数（默认1）。设置为-1表示使用所有可用核心。
   random_state: 随机种子，保证结果可复现（默认None）。
'''

# 🚩：构建模型，修改下面的代码，但是不要修改变量名：'model'

# --------------------- 修改部分 ---------------------
# 使用 XGBoost 分类器代替随机森林，XGBoost 在此类竞赛中通常性能更佳。
# 参数说明:
# n_estimators: 树的数量，即迭代次数。
# max_depth: 每棵树的最大深度，用于防止过拟合。
# learning_rate: 学习率，控制每次迭代的步长。
# use_label_encoder=False: 避免新版本 XGBoost 的警告，因为我们已经手动完成了编码。
# eval_metric='logloss': 指定评估指标为对数损失，适用于二分类问题。
model = XGBClassifier(
    n_estimators=200,          # 树的数量
    max_depth=5,               # 树的最大深度
    learning_rate=0.1,         # 学习率
    use_label_encoder=False,   # 关闭自动标签编码
    eval_metric='logloss',     # 评估指标
    n_jobs=-1,                 # 使用所有CPU核心进行并行计算
    random_state=42            # 随机种子，保证结果可复现
)
# ----------------------------------------------------

# 训练模型（不要修改代码）
model.fit(x_train, y_train)

In [ ]:
# 预测测试集并计算指标（不要修改代码）

y_pred = model.predict(x_test)
y_pred

In [ ]:
# 计算准确率（不要修改代码）

acc = accuracy_score(y_test, y_pred)
acc

In [ ]:
# 模型调参（具体参数需要根据你的模型来修改，可以回顾前面学过的内容～）
# 🚩：请修改{}内的代码
# --------------------- 修改部分 ---------------------
# 为 XGBoost 模型创建一个参数网格，用于 GridSearchCV 寻找最优参数。
# n_estimators: 尝试不同的树的数量。
# max_depth: 尝试不同的树深度，控制模型复杂度。
# learning_rate: 尝试不同的学习率。
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1],
}
# ----------------------------------------------------
# GridSearchCV
# 不要动下面的代码
# 注意：这里的 estimator 需要是我们新定义的 XGBoost 模型，而不是随机森林。
# 我们需要重新创建一个基础模型实例传给 GridSearchCV。
xgb_for_grid = XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_jobs=-1, random_state=42)

grid_search = GridSearchCV(estimator=xgb_for_grid, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2) # cv=3 为了加快演示速度
grid_search.fit(x_train, y_train)

In [ ]:
# 最佳参数和最高准确率
grid_search.best_params_, grid_search.best_score_

In [ ]:
# 计算结果集并保存
result = model.predict(test)

# 保存预测结果并保存为文件，这里要注意因为测试集存在非法值，有一部分数据被删除，所以submit 中对应索引的行也要被删除
submit['RainTomorrow'] = result

submit.to_csv("submit.csv")

In [ ]:
#运行这个Cell 下载提交工具

!wget -nv -O heywhale_submit https://cdn.kesci.com/submit_tool/v4/heywhale_submit&&chmod +x heywhale_submit

# 运行提交工具
# 把下方XXXXXXX替换为你的 Token

!./heywhale_submit -token XXXXXXX -file submit.csv  # 替换XXXXXXX；注意不可增减任何空格或其他